# KEN3450, Data Analysis, Spring 2020 

## DataMadness Submission

**Team members   :** Olivia (Olive) Kirk  &  Karim Abdel Kader.

**Respective IDs :** i6 153 141  &  i6 131 210.

**Data being used:** Gun violence dataset ([source](https://www.kaggle.com/jameslko/gun-violence-data#gun-violence-data_01-2013_03-2018.csv))

#### Questions to Answer
 - Which states, counties and congressional districts have the highest rates of shootings, both fatal and non-fatal?
 - What is the relationship between the number injured and/or killed and the "incidental_characteristics" (correlations between multiple factors)?
 - What trends change over time? (particularly the frequency of characteristics and the severity of shootings) 
 
---

**Imports**

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 400)
pd.set_option('display.max_rows', 400)

In [2]:
import warnings
#warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

def apply_styles():
    plt.rcParams['font.size'] = 12
    plt.rcParams['figure.figsize'] = (18, 6)
    plt.rcParams['lines.linewidth'] = 1

apply_styles()
sns.set(style="ticks")

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

import math

import sklearn.metrics as metrics
from scipy import stats

/home/olive/anaconda3/lib/python3.7/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
/home/olive/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/olive/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


---
---
## Part 1: Examining the data

In [3]:
dfFull = pd.read_csv("gun-violence-data_01-2013_03-2018.csv")
dfFull[-5:]

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.klfy.com/news/local/rayne-woman-cha...,False,NaN,0::Unknown,0::Unknown,Shots Fired - No Injuries,NaN,NaN,NaN,1.0,NaN,0::25,0::Adult 18+,0::Female,0::Jhkeya Tezeno,NaN,"0::Unharmed, Arrested",0::Subject-Suspect,http://www.klfy.com/news/local/rayne-woman-cha...,NaN,NaN
239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,http://www.gunviolencearchive.org/incident/108...,http://www.ksla.com/story/37854648/man-wanted-...,False,4.0,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)||Ins...",31.7537,Shop Rite,-93.0836,1.0,NaN,1::21,0::Adult 18+||1::Adult 18+,0::Male||1::Male,0::Jamal Haskett||1::Jaquarious Tyjuan Ardison,NaN,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,http://www.ksla.com/story/37854648/man-wanted-...,23.0,31.0
239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,http://www.gunviolencearchive.org/incident/108...,http://www.nola.com/crime/index.ssf/2018/04/sh...,False,2.0,0::Unknown,0::Unknown,Shot - Wounded/Injured,29.9239,NaN,-90.0442,1.0,NaN,0::21,0::Adult 18+,0::Male,NaN,NaN,0::Injured,0::Victim,http://www.nola.com/crime/index.ssf/2018/04/sh...,85.0,7.0
239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,http://www.gunviolencearchive.org/incident/108...,https://www.chron.com/news/houston-texas/houst...,False,9.0,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)",29.7201,Vanderbilt Court apartments,-95.6110,1.0,"Vic was found shot to death in car on 4/1/18, ...",0::42,0::Adult 18+,0::Male,0::Leroy Ellis,NaN,0::Killed,0::Victim,http://www.khou.com/article/news/hpd-investiga...,149.0,17.0
239676,1081940,2018-03-31,Maine,Norridgewock,434 Skowhegan Rd,2,0,http://www.gunviolencearchive.org/incident/108...,https://www.centralmaine.com/2018/03/31/police...,False,2.0,0::Unknown||1::Unknown,0::Handgun||1::Shotgun,"Shot - Dead (murder, accidental, suicide)||Sui...",44.7293,NaN,-69.7691,2.0,"ALT: US 2, shot wife then self, handgun, shotg...",0::58||1::62,0::Adult 18+||1::Adult 18+,0::Female||1::Male,0::Marie Lancaster Hale||1::William Hale,1::Significant others - current or former,0::Killed||1::Killed,0::Victim||1::Subject-Suspect,https://www.centralmaine.com/2018/03/31/police...,111.0,3.0


In [4]:
# Modify time values.
dfFull["date2"] = dfFull["date"].astype("datetime64[ns]")
dfFull.sort_values(by=['date2'], inplace = True)

dfFull[:5]

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,date2
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,2013-01-01
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,2013-01-01
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,2013-01-01
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,2013-01-05
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0,2013-01-07


In [5]:
# Reduce undesirables.
df = dfFull.drop(["address", "incident_url", "source_url", "location_description", "notes", 
                  "participant_name", "sources"], axis = 1, inplace = False)
df[:5]

,incident_id,date,state,city_or_county,n_killed,n_injured,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,longitude,n_guns_involved,participant_age,participant_age_group,participant_gender,participant_relationship,participant_status,participant_type,state_house_district,state_senate_district,date2
0,461105,2013-01-01,Pennsylvania,Mckeesport,0,4,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,-79.8559,NaN,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,NaN,NaN,2013-01-01
1,460726,2013-01-01,California,Hawthorne,1,3,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,-118.3330,NaN,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,62.0,35.0,2013-01-01
2,478855,2013-01-01,Ohio,Lorain,1,3,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,-82.1377,2.0,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,56.0,13.0,2013-01-01
3,478925,2013-01-05,Colorado,Aurora,4,0,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,-104.8020,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,40.0,28.0,2013-01-05
4,478959,2013-01-07,North Carolina,Greensboro,2,2,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,-79.9569,2.0,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,62.0,27.0,2013-01-07


In [6]:
print(df.dtypes)

incident_id                             int64
date                                   object
state                                  object
city_or_county                         object
n_killed                                int64
n_injured                               int64
incident_url_fields_missing              bool
congressional_district                float64
gun_stolen                             object
gun_type                               object
incident_characteristics               object
latitude                              float64
longitude                             float64
n_guns_involved                       float64
participant_age                        object
participant_age_group                  object
participant_gender                     object
participant_relationship               object
participant_status                     object
participant_type                       object
state_house_district                  float64
state_senate_district             

In [7]:
print("Dataframe dimensions : ", df.shape)
df.describe()

Dataframe dimensions :  (239677, 23)


,incident_id,n_killed,n_injured,congressional_district,latitude,longitude,n_guns_involved,state_house_district,state_senate_district
count,2.396770e+05,239677.000000,239677.000000,227733.000000,231754.000000,231754.000000,140226.000000,200905.000000,207342.00000
mean,5.593343e+05,0.252290,0.494007,8.001265,37.546598,-89.338348,1.372442,55.447132,20.47711
std,2.931287e+05,0.521779,0.729952,8.480835,5.130763,14.359546,4.678202,42.048117,14.20456
min,9.211400e+04,0.000000,0.000000,0.000000,19.111400,-171.429000,1.000000,1.000000,1.00000
25%,3.085450e+05,0.000000,0.000000,2.000000,33.903400,-94.158725,1.000000,21.000000,9.00000
50%,5.435870e+05,0.000000,0.000000,5.000000,38.570600,-86.249600,1.000000,47.000000,19.00000
75%,8.172280e+05,0.000000,1.000000,10.000000,41.437375,-80.048625,1.000000,84.000000,30.00000
max,1.083472e+06,50.000000,53.000000,53.000000,71.336800,97.433100,400.000000,901.000000,94.00000


---
---
## Part 2: Cleaning the data

### Handling missing values

In [8]:
# Missing value summary
print("Dataframe dimensions : ", df.shape)
print()
empty = df.isnull().sum()
print(empty)
print("SUM:                           ", empty.sum())

Dataframe dimensions :  (239677, 23)

incident_id                         0
date                                0
state                               0
city_or_county                      0
n_killed                            0
n_injured                           0
incident_url_fields_missing         0
congressional_district          11944
gun_stolen                      99498
gun_type                        99451
incident_characteristics          326
latitude                         7923
longitude                        7923
n_guns_involved                 99451
participant_age                 92298
participant_age_group           42119
participant_gender              36362
participant_relationship       223903
participant_status              27626
participant_type                24863
state_house_district            38772
state_senate_district           32335
date2                               0
dtype: int64
SUM:                            844794


In [9]:
# Restore...
df = dfFull.drop(["address", "incident_url", "source_url", "location_description", "notes", 
                  "participant_name", "sources"], axis = 1, inplace = False)

# Reduce...
df.drop(["participant_age", "participant_relationship"], 
        axis = 1, inplace = True)

# Reveal...
print("Dataframe dimensions before : ", df.shape)
df.dropna(inplace = True)
print("Dataframe dimensions after  : ", df.shape)

empty = df.isnull().sum()
print("Emptiness safety check, should be 0 :", empty.sum())

Dataframe dimensions before :  (239677, 21)
Dataframe dimensions after  :  (90628, 21)
Emptiness safety check, should be 0 : 0


In [10]:
# So that we can edit "wrong" values manually.
name = "droppedData.csv"
df.to_csv(name, index=True)

### Process the remainder

In [11]:
df[:5]

,incident_id,date,state,city_or_county,n_killed,n_injured,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,longitude,n_guns_involved,participant_age_group,participant_gender,participant_status,participant_type,state_house_district,state_senate_district,date2
2,478855,2013-01-01,Ohio,Lorain,1,3,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,-82.1377,2.0,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,56.0,13.0,2013-01-01
4,478959,2013-01-07,North Carolina,Greensboro,2,2,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,-79.9569,2.0,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,62.0,27.0,2013-01-07
6,479363,2013-01-19,New Mexico,Albuquerque,5,0,False,1.0,0::Unknown||1::Unknown,0::22 LR||1::223 Rem [AR-15],"Shot - Dead (murder, accidental, suicide)||Mas...",34.9791,-106.7160,2.0,0::Adult 18+||1::Adult 18+||2::Child 0-11||3::...,0::Male||1::Female||2::Male||3::Female||4::Fem...,0::Killed||1::Killed||2::Killed||3::Killed||4:...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,10.0,14.0,2013-01-19
10,491674,2013-01-23,Tennessee,Chattanooga,1,3,False,3.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",35.0221,-85.2697,1.0,0::Adult 18+,0::Male||1::Male||2::Male||3::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,28.0,10.0,2013-01-23
11,479413,2013-01-25,Missouri,Saint Louis,1,3,False,1.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",38.7067,-90.2494,1.0,0::Adult 18+,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,76.0,4.0,2013-01-25


In [12]:
# Reduce useless columns.
print(df["incident_url_fields_missing"].value_counts().shape)
df["incident_url_fields_missing"].value_counts()

(1,)


False    90628
Name: incident_url_fields_missing, dtype: int64

In [13]:
# Tidy me.

def process(dataframe, OGcol, newHead, verbose):

    size1 = dataframe.shape[1]
    if verbose:
        print(dataframe[OGcol].value_counts().shape)
        print(dataframe[OGcol].value_counts())
        print()

    # Fix the genre identification.
    tmpDict = {}
    for i, e in enumerate(dataframe[OGcol]):
        #print("current index is", i, "with element", e)
        temp = e.split("|")
        for t in temp:
            tmp = t.split(":")
            #print(i, tmp)
            tmpDict[tmp[-1]] = 0

    tmpList = []
    for x in tmpDict.items():
        col = newHead + ":"+x[0]
        dataframe[col] = False
        tmpList.append(x[0])
    
    if verbose:
        print(tmpList)
        print()

    for c in tmpList:
        col = newHead +":"+c
        dataframe[col] = 0
    
    
    holdme = 0
    for e, col in enumerate(dataframe.columns): 
        #print(col)
        if col == OGcol:
            #print(col, OGcol, holdme)
            holdme = e
            break
        
    for r in dataframe.iterrows():
        # DF ID: print(r[0])
        i = r[0]
        #print(r[1][holdme])
        e = r[1][holdme]

        temp = e.split("|")
        for t in temp:
            tmp = t.split(":")
            #print(i, tmp)
            col = newHead+":"+tmp[-1]
            if len(tmp) == 0:
                print("AT INDEX", i)
            for t in tmp:
                dataframe[col][i] = 1
    
    print(dataframe.shape[1]-size1, "rows add, 2 rows removed--")
    dataframe.drop([OGcol, newHead+":"], axis=1, inplace=True)
    if verbose:
        dataframe[:5]

In [14]:
print("Processing stolen status...")
process(df, "gun_stolen", "stolenStatus", False)
print("Processing type status...")
process(df, "gun_type", "gunType", False)
print("Processing incident characteristics...")
process(df, "incident_characteristics", "incident_characteristics", False)
print("Processing age status...")
process(df, "participant_age_group", "ageGroup", False)
print("Processing gender status...")
process(df, "participant_gender", "gender", False)

Processing stolen status...


/home/olive/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4 rows add, 2 rows removed--
Processing type status...
28 rows add, 2 rows removed--
Processing incident characteristics...
110 rows add, 2 rows removed--
Processing age status...
4 rows add, 2 rows removed--
Processing gender status...
3 rows add, 2 rows removed--


In [15]:
df[:4]

,incident_id,date,state,city_or_county,n_killed,n_injured,incident_url_fields_missing,congressional_district,latitude,longitude,n_guns_involved,participant_status,participant_type,state_house_district,state_senate_district,date2,stolenStatus:Unknown,stolenStatus:Not-stolen,stolenStatus:Stolen,gunType:Unknown,gunType:Handgun,gunType:22 LR,gunType:223 Rem [AR-15],gunType:Shotgun,gunType:9mm,gunType:45 Auto,gunType:12 gauge,gunType:7.62 [AK-47],gunType:40 SW,gunType:44 Mag,gunType:Other,gunType:38 Spl,gunType:380 Auto,gunType:410 gauge,gunType:32 Auto,gunType:308 Win,gunType:Rifle,gunType:357 Mag,gunType:16 gauge,gunType:30-30 Win,gunType:25 Auto,gunType:20 gauge,gunType:30-06 Spr,gunType:300 Win,gunType:10mm,gunType:28 gauge,incident_characteristics:Shot - Wounded/Injured,"incident_characteristics:Shot - Dead (murder, accidental, suicide)",incident_characteristics:Shots Fired - No Injuries,incident_characteristics:Bar/club incident - in or around establishment,incident_characteristics:Suicide^,incident_characteristics:Murder/Suicide,incident_characteristics:Attempted Murder/Suicide (one variable unsuccessful),incident_characteristics:Domestic Violence,"incident_characteristics:Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)","incident_characteristics:Drive-by (car to street, car to car)",incident_characteristics:Gang involvement,incident_characteristics:Home Invasion,incident_characteristics:Home Invasion - Resident killed,incident_characteristics:Officer Involved Incident,incident_characteristics:Officer Involved Shooting - Officer shot,incident_characteristics:Officer Involved Shooting - Officer killed,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator shot,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator suicide at standoff,"incident_characteristics:Spree Shooting (multiple victims, multiple locations)",incident_characteristics:Possession of gun by felon or prohibited person,incident_characteristics:Institution/Group/Business,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator killed,incident_characteristics:Animal shot/killed,incident_characteristics:Home Invasion - Resident injured,incident_characteristics:Drug involvement,incident_characteristics:Armed robbery with injury/death and/or evidence of DGU found,"incident_characteristics:Mass Murder (4+ deceased victims excluding the subject/suspect/perpetrator , one location)",incident_characteristics:Concealed Carry License - Perpetrator,incident_characteristics:Possession (gun(s) found during commission of other crimes),incident_characteristics:Stolen/Illegally owned gun{s} recovered during arrest/warrant,incident_characteristics:School Incident,incident_characteristics:School Shooting - university/college,incident_characteristics:Kidnapping/abductions/hostage,incident_characteristics:Car-jacking,incident_characteristics:Hate crime,incident_characteristics:House party,incident_characteristics:Defensive Use,"incident_characteristics:Defensive Use - Crime occurs, victim shoots subject/suspect/perpetrator",incident_characteristics:Defensive Use - Victim stops crime,incident_characteristics:Workplace shooting (disgruntled employee),"incident_characteristics:Assault weapon (AR-15, AK-47, and ALL variants defined by law enforcement)",incident_characteristics:Accidental Shooting,incident_characteristics:Accidental Shooting - Injury,incident_characteristics:Accidental/Negligent Discharge,incident_characteristics:Non-Shooting Incident,incident_characteristics:Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator ),incident_characteristics:ATF/LE Confiscation/Raid/Arrest,incident_characteristics:Unlawful purchase/sale,incident_characteristics:Accidental Shooting - Death,incident_characteristics:BB/Pellet/Replica gun,incident_characteristics:Brandishing/flourishing/open carry/lost/found,incident_characteristics

In [16]:
# Reduce but maybe remove these later...
#df.drop(["gun_stolen"], axis = 1, inplace = True)
#df.drop(["gun_type"], axis = 1, inplace = True)
#df.drop(["incident_characteristics"], axis = 1, inplace = True)
df.drop(["participant_status"], axis = 1, inplace = True)
df.drop(["participant_type"], axis = 1, inplace = True)

In [17]:
df.drop(["incident_id", "incident_url_fields_missing"], axis = 1, inplace = True)
df[:5]

,date,state,city_or_county,n_killed,n_injured,congressional_district,latitude,longitude,n_guns_involved,state_house_district,state_senate_district,date2,stolenStatus:Unknown,stolenStatus:Not-stolen,stolenStatus:Stolen,gunType:Unknown,gunType:Handgun,gunType:22 LR,gunType:223 Rem [AR-15],gunType:Shotgun,gunType:9mm,gunType:45 Auto,gunType:12 gauge,gunType:7.62 [AK-47],gunType:40 SW,gunType:44 Mag,gunType:Other,gunType:38 Spl,gunType:380 Auto,gunType:410 gauge,gunType:32 Auto,gunType:308 Win,gunType:Rifle,gunType:357 Mag,gunType:16 gauge,gunType:30-30 Win,gunType:25 Auto,gunType:20 gauge,gunType:30-06 Spr,gunType:300 Win,gunType:10mm,gunType:28 gauge,incident_characteristics:Shot - Wounded/Injured,"incident_characteristics:Shot - Dead (murder, accidental, suicide)",incident_characteristics:Shots Fired - No Injuries,incident_characteristics:Bar/club incident - in or around establishment,incident_characteristics:Suicide^,incident_characteristics:Murder/Suicide,incident_characteristics:Attempted Murder/Suicide (one variable unsuccessful),incident_characteristics:Domestic Violence,"incident_characteristics:Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)","incident_characteristics:Drive-by (car to street, car to car)",incident_characteristics:Gang involvement,incident_characteristics:Home Invasion,incident_characteristics:Home Invasion - Resident killed,incident_characteristics:Officer Involved Incident,incident_characteristics:Officer Involved Shooting - Officer shot,incident_characteristics:Officer Involved Shooting - Officer killed,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator shot,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator suicide at standoff,"incident_characteristics:Spree Shooting (multiple victims, multiple locations)",incident_characteristics:Possession of gun by felon or prohibited person,incident_characteristics:Institution/Group/Business,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator killed,incident_characteristics:Animal shot/killed,incident_characteristics:Home Invasion - Resident injured,incident_characteristics:Drug involvement,incident_characteristics:Armed robbery with injury/death and/or evidence of DGU found,"incident_characteristics:Mass Murder (4+ deceased victims excluding the subject/suspect/perpetrator , one location)",incident_characteristics:Concealed Carry License - Perpetrator,incident_characteristics:Possession (gun(s) found during commission of other crimes),incident_characteristics:Stolen/Illegally owned gun{s} recovered during arrest/warrant,incident_characteristics:School Incident,incident_characteristics:School Shooting - university/college,incident_characteristics:Kidnapping/abductions/hostage,incident_characteristics:Car-jacking,incident_characteristics:Hate crime,incident_characteristics:House party,incident_characteristics:Defensive Use,"incident_characteristics:Defensive Use - Crime occurs, victim shoots subject/suspect/perpetrator",incident_characteristics:Defensive Use - Victim stops crime,incident_characteristics:Workplace shooting (disgruntled employee),"incident_characteristics:Assault weapon (AR-15, AK-47, and ALL variants defined by law enforcement)",incident_characteristics:Accidental Shooting,incident_characteristics:Accidental Shooting - Injury,incident_characteristics:Accidental/Negligent Discharge,incident_characteristics:Non-Shooting Incident,incident_characteristics:Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator ),incident_characteristics:ATF/LE Confiscation/Raid/Arrest,incident_characteristics:Unlawful purchase/sale,incident_characteristics:Accidental Shooting - Death,incident_characteristics:BB/Pellet/Replica gun,incident_characteristics:Brandishing/flourishing/open carry/lost/found,incident_characteristics:Child Involved Incident,incident_characteristics:Child injured self,inciden

In [18]:
# Reform the date.
for r in df.iterrows():
    i = r[0]
    #print(i)
    df["date"][i] = str(df["date"][i])[:7]
    #dfFull["date2"][0] = str(dfFull["date"][0])[:7]

/home/olive/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
df.describe()

,n_killed,n_injured,congressional_district,latitude,longitude,n_guns_involved,state_house_district,state_senate_district,stolenStatus:Unknown,stolenStatus:Not-stolen,stolenStatus:Stolen,gunType:Unknown,gunType:Handgun,gunType:22 LR,gunType:223 Rem [AR-15],gunType:Shotgun,gunType:9mm,gunType:45 Auto,gunType:12 gauge,gunType:7.62 [AK-47],gunType:40 SW,gunType:44 Mag,gunType:Other,gunType:38 Spl,gunType:380 Auto,gunType:410 gauge,gunType:32 Auto,gunType:308 Win,gunType:Rifle,gunType:357 Mag,gunType:16 gauge,gunType:30-30 Win,gunType:25 Auto,gunType:20 gauge,gunType:30-06 Spr,gunType:300 Win,gunType:10mm,gunType:28 gauge,incident_characteristics:Shot - Wounded/Injured,"incident_characteristics:Shot - Dead (murder, accidental, suicide)",incident_characteristics:Shots Fired - No Injuries,incident_characteristics:Bar/club incident - in or around establishment,incident_characteristics:Suicide^,incident_characteristics:Murder/Suicide,incident_characteristics:Attempted Murder/Suicide (one variable unsuccessful),incident_characteristics:Domestic Violence,"incident_characteristics:Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)","incident_characteristics:Drive-by (car to street, car to car)",incident_characteristics:Gang involvement,incident_characteristics:Home Invasion,incident_characteristics:Home Invasion - Resident killed,incident_characteristics:Officer Involved Incident,incident_characteristics:Officer Involved Shooting - Officer shot,incident_characteristics:Officer Involved Shooting - Officer killed,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator shot,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator suicide at standoff,"incident_characteristics:Spree Shooting (multiple victims, multiple locations)",incident_characteristics:Possession of gun by felon or prohibited person,incident_characteristics:Institution/Group/Business,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator killed,incident_characteristics:Animal shot/killed,incident_characteristics:Home Invasion - Resident injured,incident_characteristics:Drug involvement,incident_characteristics:Armed robbery with injury/death and/or evidence of DGU found,"incident_characteristics:Mass Murder (4+ deceased victims excluding the subject/suspect/perpetrator , one location)",incident_characteristics:Concealed Carry License - Perpetrator,incident_characteristics:Possession (gun(s) found during commission of other crimes),incident_characteristics:Stolen/Illegally owned gun{s} recovered during arrest/warrant,incident_characteristics:School Incident,incident_characteristics:School Shooting - university/college,incident_characteristics:Kidnapping/abductions/hostage,incident_characteristics:Car-jacking,incident_characteristics:Hate crime,incident_characteristics:House party,incident_characteristics:Defensive Use,"incident_characteristics:Defensive Use - Crime occurs, victim shoots subject/suspect/perpetrator",incident_characteristics:Defensive Use - Victim stops crime,incident_characteristics:Workplace shooting (disgruntled employee),"incident_characteristics:Assault weapon (AR-15, AK-47, and ALL variants defined by law enforcement)",incident_characteristics:Accidental Shooting,incident_characteristics:Accidental Shooting - Injury,incident_characteristics:Accidental/Negligent Discharge,incident_characteristics:Non-Shooting Incident,incident_characteristics:Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator ),incident_characteristics:ATF/LE Confiscation/Raid/Arrest,incident_characteristics:Unlawful purchase/sale,incident_characteristics:Accidental Shooting - Death,incident_characteristics:BB/Pellet/Replica gun,incident_characteristics:Brandishing/flourishing/open carry/lost/found,incident_characteristics:Child Involved Incident,incident_characteristics:Child injured self,incident_characteristics:Gun range/gun 

In [20]:
print("Dimensions:", df.shape)
df[:5]

Dimensions: (90628, 156)


,date,state,city_or_county,n_killed,n_injured,congressional_district,latitude,longitude,n_guns_involved,state_house_district,state_senate_district,date2,stolenStatus:Unknown,stolenStatus:Not-stolen,stolenStatus:Stolen,gunType:Unknown,gunType:Handgun,gunType:22 LR,gunType:223 Rem [AR-15],gunType:Shotgun,gunType:9mm,gunType:45 Auto,gunType:12 gauge,gunType:7.62 [AK-47],gunType:40 SW,gunType:44 Mag,gunType:Other,gunType:38 Spl,gunType:380 Auto,gunType:410 gauge,gunType:32 Auto,gunType:308 Win,gunType:Rifle,gunType:357 Mag,gunType:16 gauge,gunType:30-30 Win,gunType:25 Auto,gunType:20 gauge,gunType:30-06 Spr,gunType:300 Win,gunType:10mm,gunType:28 gauge,incident_characteristics:Shot - Wounded/Injured,"incident_characteristics:Shot - Dead (murder, accidental, suicide)",incident_characteristics:Shots Fired - No Injuries,incident_characteristics:Bar/club incident - in or around establishment,incident_characteristics:Suicide^,incident_characteristics:Murder/Suicide,incident_characteristics:Attempted Murder/Suicide (one variable unsuccessful),incident_characteristics:Domestic Violence,"incident_characteristics:Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)","incident_characteristics:Drive-by (car to street, car to car)",incident_characteristics:Gang involvement,incident_characteristics:Home Invasion,incident_characteristics:Home Invasion - Resident killed,incident_characteristics:Officer Involved Incident,incident_characteristics:Officer Involved Shooting - Officer shot,incident_characteristics:Officer Involved Shooting - Officer killed,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator shot,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator suicide at standoff,"incident_characteristics:Spree Shooting (multiple victims, multiple locations)",incident_characteristics:Possession of gun by felon or prohibited person,incident_characteristics:Institution/Group/Business,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator killed,incident_characteristics:Animal shot/killed,incident_characteristics:Home Invasion - Resident injured,incident_characteristics:Drug involvement,incident_characteristics:Armed robbery with injury/death and/or evidence of DGU found,"incident_characteristics:Mass Murder (4+ deceased victims excluding the subject/suspect/perpetrator , one location)",incident_characteristics:Concealed Carry License - Perpetrator,incident_characteristics:Possession (gun(s) found during commission of other crimes),incident_characteristics:Stolen/Illegally owned gun{s} recovered during arrest/warrant,incident_characteristics:School Incident,incident_characteristics:School Shooting - university/college,incident_characteristics:Kidnapping/abductions/hostage,incident_characteristics:Car-jacking,incident_characteristics:Hate crime,incident_characteristics:House party,incident_characteristics:Defensive Use,"incident_characteristics:Defensive Use - Crime occurs, victim shoots subject/suspect/perpetrator",incident_characteristics:Defensive Use - Victim stops crime,incident_characteristics:Workplace shooting (disgruntled employee),"incident_characteristics:Assault weapon (AR-15, AK-47, and ALL variants defined by law enforcement)",incident_characteristics:Accidental Shooting,incident_characteristics:Accidental Shooting - Injury,incident_characteristics:Accidental/Negligent Discharge,incident_characteristics:Non-Shooting Incident,incident_characteristics:Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator ),incident_characteristics:ATF/LE Confiscation/Raid/Arrest,incident_characteristics:Unlawful purchase/sale,incident_characteristics:Accidental Shooting - Death,incident_characteristics:BB/Pellet/Replica gun,incident_characteristics:Brandishing/flourishing/open carry/lost/found,incident_characteristics:Child Involved Incident,incident_characteristics:Child injured self,inciden

In [21]:
print(df.dtypes)

date                                                                                                                                     object
state                                                                                                                                    object
city_or_county                                                                                                                           object
n_killed                                                                                                                                  int64
n_injured                                                                                                                                 int64
congressional_district                                                                                                                  float64
latitude                                                                                                                                

### Aggregate for visualisation (1)

In [22]:
# [0] is 2019, [1] is 2010.
statePoP = {}

statePoP["California"] = [39512223, 37254523]
statePoP["Texas"] = [28995881, 25145561]
statePoP["Florida"] = [21477737, 18801310 ]
statePoP["New York"] = [19453561, 19378102]
statePoP["Pennsylvania"] = [12801989, 12702379]
statePoP["Illinois"] = [12671821, 12830632]
statePoP["Ohio"] = [11689100, 11536504] 
statePoP["Georgia"] = [10617423, 9687653]
statePoP["North Carolina"] = [10488084, 9535483]
statePoP["Michigan"] = [9986857, 9883640] 
statePoP["New Jersey"] = [8882190, 8791894] 
statePoP["Virginia"] = [8535519, 8001024]
statePoP["Washington"] = [7614893, 6724540] 
statePoP["Arizona"] = [7278717, 6392017]
statePoP["Massachusetts"] = [6949503, 6547629]
statePoP["Tennessee"] = [6833174, 6346105]
statePoP["Indiana"] = [6732219, 6483802]
statePoP["Missouri"] = [6137428, 5988927]
statePoP["Maryland"] = [6045680, 5773552]
statePoP["Wisconsin"] = [5822434, 5686986]
statePoP["Colorado"] = [5758736, 5029196]
statePoP["Minnesota"] = [5639632, 5303925]
statePoP["South Carolina"] = [5148714, 4625364]
statePoP["Alabama"] = [4903185, 4779736]
statePoP["Louisiana"] = [4648794, 4533372]
statePoP["Kentucky"] = [4467673, 4339367]
statePoP["Oregon"] = [4217737, 3831074]
statePoP["Oklahoma"] = [3956971, 3751351]
statePoP["Connecticut"] = [3565287, 3574097]
statePoP["Utah"] = [3205958, 2763885]
statePoP["Iowa"] = [3155070, 3046355]
statePoP["Nevada"] = [3080156, 2700551]
statePoP["Arkansas"] = [3017825, 2915918]
statePoP["Mississippi"] = [2976149, 2967297]
statePoP["Kansas"] = [2913314, 2853118]
statePoP["New Mexico"] = [2096829, 2059179]
statePoP["Nebraska"] = [1934408, 1826341]
statePoP["West Virginia"] = [1792065, 1852994]
statePoP["Idaho"] = [1787147, 1567582]
statePoP["Hawaii"] = [1415872, 1360301]
statePoP["New Hampshire"] = [1359711, 1316470]
statePoP["Maine"] = [1344212, 1328361]
statePoP["Montana"] = [1068778, 989415]
statePoP["Rhode Island"] = [1059361, 1052567]
statePoP["Delaware"] = [973764, 897934]
statePoP["South Dakota"] = [884659, 814180]
statePoP["North Dakota"] = [762062, 672591]
statePoP["Alaska"] = [731545, 710231]
statePoP["Vermont"] = [623989, 625741]
statePoP["District of Columbia"] = [705749, 601723]
statePoP["Wyoming"] = [578759, 563626]

In [23]:
df2 = pd.read_csv("out12.csv")
codes = {}

for r in df2.iterrows():
    i = r[0]
    codes[df2["state"][i]] = df2["code"][i]
codes["Vermont"] = "VT"

for c in codes:
    1 == 1
    print(c, codes[c])

Illinois IL
California CA
Florida FL
Ohio OH
New York NY
Texas TX
Georgia GA
Louisiana LA
Indiana IN
Tennessee TN
Missouri MO
North Carolina NC
Michigan MI
South Carolina SC
New Jersey NJ
Virginia VA
Alabama AL
Maryland MD
Washington WA
Wisconsin WI
Mississippi MS
Connecticut CT
Oklahoma OK
Arkansas AR
Colorado CO
Oregon OR
Arizona AZ
Iowa IA
Pennsylvania PA
Kansas KS
Delaware DE
West Virginia WV
Nevada NV
New Mexico NM
Massachusetts MA
Utah UT
Maine ME
Rhode Island RI
Kentucky KY
Idaho ID
North Dakota ND
South Dakota SD
Hawaii HI
Wyoming WY
New Hampshire NH
Montana MT
Alaska AK
District of Columbia DC
Minnesota MN
Nebraska NE
Vermont VT


In [24]:
def getRow(dataframe, col, val):
    a = dataframe.loc[dataframe[col] == val]
    #print((a), "\n")
    #print((a.index), "\n")
    #print((a.index.tolist()), "\n")
    
    fin = a.index.tolist()
    print("Number of matching rows for (", col, ",", val, "):", len(fin))
    #print(fin)
    
    return fin

In [25]:
print(dfFull["state"].value_counts().shape)
print(dfFull["state"].value_counts())
listofStates = dfFull["state"].value_counts().index.tolist()
#print(listofStates)

(51,)
Illinois                17556
California              16306
Florida                 15029
Texas                   13577
Ohio                    10244
New York                 9712
Pennsylvania             8929
Georgia                  8925
North Carolina           8739
Louisiana                8103
Tennessee                7626
South Carolina           6939
Missouri                 6631
Michigan                 6136
Massachusetts            5981
Virginia                 5949
Indiana                  5852
Maryland                 5798
Alabama                  5471
New Jersey               5387
Wisconsin                4787
Kentucky                 4157
Mississippi              3599
Oklahoma                 3455
Washington               3434
Colorado                 3201
District of Columbia     3195
Connecticut              3067
Arkansas                 2842
Iowa                     2517
Minnesota                2408
Arizona                  2328
Oregon                   2286
Kans

In [26]:
for i in statePoP.keys():
    if i not in listofStates:
        print(i)
print()
for i in listofStates:
    if i not in statePoP.keys():
        print(i)
        
print(len(statePoP))


51


In [27]:
for i in codes:
    if i not in listofStates:
        print(i)
print()
for i in listofStates:
    if i not in codes:
        print(i)
        
print(len(codes))
print(len(listofStates))


51
51


In [28]:
print(df["city_or_county"].value_counts().shape)
df["city_or_county"].value_counts()
listofCoC = df["city_or_county"].value_counts().index.tolist()
#print(listofCoC)

(8432,)


In [29]:
skip = ["date", "date2", "state", "city_or_county", "congressional_district", "latitude", "longitude", 
        "state_house_district", "state_senate_district"]

keep = ["n_killed", "n_injured", "n_guns_involved"]

In [30]:
# Stack up with respect to location
df1 = pd.DataFrame(listofStates, columns = ["state"]) 

# Define additional columns in new dataframe.
df1["code"] = ""

df1["entries"] = 0

for col in df.columns: 
    if col in keep:
        df1[col] = 0
        
        
df1["noHarm"] = 0
df1["incidentsWithDeath"] = 0
df1["incidentsWithInjury"] = 0
df1["multiGunCount"] = 0
df1["injWithoutDeath"] = 0
df1["noHarmProp"] = 0.0
df1["incidentsWithDeathProp"] = 0.0
df1["incidentsWithInjuryProp"] = 0.0
df1["multiGunCountProp"] = 0.0
df1["injWithoutDeathProp"] = 0.0

df1["killAvg"] = 0.0
df1["injAvg"] = 0.0
df1["nogunAvg"] = 0.0
df1["killAvgNorm19"] = 0.0
df1["injAvgNorm19"] = 0.0
df1["nogunAvgNorm19"] = 0.0
df1["killAvgNorm10"] = 0.0
df1["injAvgNorm10"] = 0.0
df1["nogunAvgNorm10"] = 0.0

#a = getRow(df1, "state", "Ohio")
# Iterate over states.
for s in listofStates:    
    i = getRow(df1, "state", s)[0]
    match = getRow(dfFull, "state", s)
    
    df1["code"][i] = codes[s]
    
    # Iterate over matching rows.
    for m in match:
        # Add to total incidents in that state.
        df1["entries"][i] = df1["entries"][i] + 1

        if dfFull["n_killed"][m] == 0:
            if dfFull["n_injured"][m] == 0:
                df1["noHarm"][i] = df1["noHarm"][i] + 1
            else:
                df1["injWithoutDeath"][i] = df1["injWithoutDeath"][i] + 1
        if dfFull["n_killed"][m] > 0:
            df1["incidentsWithDeath"][i] = df1["incidentsWithDeath"][i] + 1
        if dfFull["n_injured"][m] > 0:
            df1["incidentsWithInjury"][i] = df1["incidentsWithInjury"][i] + 1
        if dfFull["n_guns_involved"][m] > 1:
            df1["multiGunCount"][i] = df1["multiGunCount"][i] + 1
        
        # Stack other values.
        for col in dfFull.columns: 
            if col in keep:
                df1[col][i] = df1[col][i] + dfFull[col][m]
        
    
    # Take averages.
    df1["killAvg"][i] = df1["n_killed"][i] / df1["entries"][i]
    df1["injAvg"][i] = df1["n_injured"][i] / df1["entries"][i]
    #df1["nogunAvg"][i] = df1["n_guns_involved"][i] / df1["entries"][i]
    
    df1["noHarmProp"][i] = df1["noHarm"][i]/df1["entries"][i]
    df1["incidentsWithDeathProp"][i] = df1["incidentsWithDeath"][i]/df1["entries"][i]
    df1["incidentsWithInjuryProp"][i] = df1["incidentsWithInjury"][i]/df1["entries"][i]
    df1["multiGunCountProp"][i] = df1["multiGunCount"][i]/df1["entries"][i]
    df1["injWithoutDeathProp"][i] = df1["injWithoutDeath"][i]/df1["entries"][i]
    
    # Make normals
    df1["killAvgNorm19"][i] = df1["n_killed"][i] / statePoP[s][0]
    df1["injAvgNorm19"][i] = df1["n_injured"][i] / statePoP[s][0]
    #df1["nogunAvgNorm19"][i] = df1["n_guns_involved"][i] / statePoP[s][0]
    
    df1["killAvgNorm10"][i] = df1["n_killed"][i] / statePoP[s][1]
    df1["injAvgNorm10"][i] = df1["n_injured"][i] / statePoP[s][1]
    #df1["nogunAvgNorm10"][i] = df1["n_guns_involved"][i] / statePoP[s][1]

"""for r in df.iterrows():
    # DF ID: print(r[0])
    i = r[0]
    #print(r[1][1])
    e = r[1][1]
    
    for t in tmp:
        df[col][i] = 1"""

df1[:10]

Number of matching rows for ( state , Illinois ): 1
Number of matching rows for ( state , Illinois ): 17556


/home/olive/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/olive/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Number of matching rows for ( state , California ): 1
Number of matching rows for ( state , California ): 16306
Number of matching rows for ( state , Florida ): 1
Number of matching rows for ( state , Florida ): 15029
Number of matching rows for ( state , Texas ): 1
Number of matching rows for ( state , Texas ): 13577
Number of matching rows for ( state , Ohio ): 1
Number of matching rows for ( state , Ohio ): 10244
Number of matching rows for ( state , New York ): 1
Number of matching rows for ( state , New York ): 9712
Number of matching rows for ( state , Pennsylvania ): 1
Number of matching rows for ( state , Pennsylvania ): 8929
Number of matching rows for ( state , Georgia ): 1
Number of matching rows for ( state , Georgia ): 8925
Number of matching rows for ( state , North Carolina ): 1
Number of matching rows for ( state , North Carolina ): 8739
Number of matching rows for ( state , Louisiana ): 1
Number of matching rows for ( state , Louisiana ): 8103
Number of matching rows f

,state,code,entries,n_killed,n_injured,n_guns_involved,noHarm,incidentsWithDeath,incidentsWithInjury,multiGunCount,injWithoutDeath,noHarmProp,incidentsWithDeathProp,incidentsWithInjuryProp,multiGunCountProp,injWithoutDeathProp,killAvg,injAvg,nogunAvg,killAvgNorm19,injAvgNorm19,nogunAvgNorm19,killAvgNorm10,injAvgNorm10,nogunAvgNorm10
0,Illinois,IL,17556,3409,13514,NaN,4014,3162,11023,648,10380,0.228640,0.180109,0.627877,0.036910,0.591251,0.194179,0.769765,0.0,0.000269,0.001066,0.0,0.000266,0.001053,0.0
1,California,CA,16306,5562,7644,NaN,6335,4972,5870,940,4999,0.388507,0.304918,0.359990,0.057647,0.306574,0.341101,0.468784,0.0,0.000141,0.000193,0.0,0.000149,0.000205,0.0
2,Florida,FL,15029,3909,7072,NaN,6680,3354,5597,862,4995,0.444474,0.223169,0.372413,0.057356,0.332357,0.260097,0.470557,0.0,0.000182,0.000329,0.0,0.000208,0.000376,0.0
3,Texas,TX,13577,5046,6106,NaN,4947,4350,4935,616,4280,0.364366,0.320395,0.363482,0.045371,0.315239,0.371658,0.449731,0.0,0.000174,0.000211,0.0,0.000201,0.000243,0.0
4,Ohio,OH,10244,2508,5703,NaN,3554,2244,4780,647,4446,0.346935,0.219055,0.466615,0.063159,0.434010,0.244826,0.556716,0.0,0.000215,0.000488,0.0,0.000217,0.000494,0.0
5,New York,NY,9712,1705,5059,NaN,4298,1559,4127,646,3855,0.442545,0.160523,0.424938,0.066516,0.396932,0.175556,0.520902,0.0,0.000088,0.000260,0.0,0.000088,0.000261,0.0
6,Pennsylvania,PA,8929,2395,5056,NaN,2973,2126,4171,519,3830,0.332960,0.238101,0.467130,0.058125,0.428939,0.268227,0.566245,0.0,0.000187,0.000395,0.0,0.000189,0.000398,0.0
7,Georgia,GA,8925,2456,4056,NaN,3811,2183,3305,392,2931,0.427003,0.244594,0.370308,0.043922,0.328403,0.275182,0.454454,0.0,0.000231,0.000382,0.0,0.000254,0.000419,0.0
8,North Carolina,NC,8739,2225,4629,NaN,3227,2015,3809,356,3497,0.369264,0.230576,0.435862,0.040737,0.400160,0.254606,0.529694,0.0,0.000212,0.000441,0.0,0.000233,0.000485,0.0
9,Louisiana,LA,8103,2179,4398,NaN,2847,1976,3582,319,3280,0.351351,0.243860,0.442058,0.039368,0.404788,0.268913,0.542762,0.0,0.000469,0.000946,0.0,0.000481,0.000970,0.0


### Aggregate for visualisation (2)

In [31]:
dfFull[:5]

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,date2
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,2013-01-01
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,2013-01-01
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,2013-01-01
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,2013-01-05
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0,2013-01-07


### Aggregate for visualisation (3)

In [32]:
# Make timesteps.
dates = []
for y in range(2013,2019):
    for m in range(1,13):
        if y == 2018 and m == 4:
            break
        if m < 10:
            dates.append(str(y) + "-0" + str(m))
        else:
            dates.append(str(y) + "-" + str(m))
            
print(dates)

['2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03']


In [33]:
# Stats over time.      
# Stack up with respect to date
df3 = pd.DataFrame(dates, columns = ["date"]) 

# Define additional columns in new dataframe.
df3["entries"] = 0
df3["noHarm"] = 0
df3["incidentsWithDeath"] = 0
df3["incidentsWithInjury"] = 0
df3["multiGunCount"] = 0
df3["noHarmProp"] = 0.0
df3["incidentsWithDeathProp"] = 0.0
df3["incidentsWithInjuryProp"] = 0.0
df3["multiGunCountProp"] = 0.0

df3["killAvg"] = 0.0
df3["injAvg"] = 0.0
df3["nogunAvg"] = 0.0

for col in df.columns: 
    if col in keep:
        df3[col] = 0
    elif col[:25]== "incident_characteristics:":
        df3[col] = 0

# Iterate over dates.
for d in dates:
    i = getRow(df3, "date", d)[0]
    match = getRow(df, "date", d)
    
    # Iterate over matching rows.
    for m in match:
        # Add to total incidents in that state.
        df3["entries"][i] = df3["entries"][i] + 1
        
        if df["n_injured"][m] == 0 and df["n_killed"][m] == 0:
            df3["noHarm"][i] = df3["noHarm"][i] + 1
        if df["n_killed"][m] > 0:
            df3["incidentsWithDeath"][i] = df3["incidentsWithDeath"][i] + 1
        if df["n_injured"][m] > 0:
            df3["incidentsWithInjury"][i] = df3["incidentsWithInjury"][i] + 1
        if df["n_guns_involved"][m] > 1:
            df3["multiGunCount"][i] = df3["multiGunCount"][i] + 1
        
        # Stack other values.
        for col in df.columns: 
            if col in keep:
                df3[col][i] = df3[col][i] + df[col][m]
            elif col[:25]== "incident_characteristics:":
                df3[col][i] = df3[col][i] + df[col][m]
        
    
    # Take averages.
    df3["killAvg"][i] = df3["n_killed"][i] / df3["entries"][i]
    df3["injAvg"][i] = df3["n_injured"][i] / df3["entries"][i]
    df3["nogunAvg"][i] = df3["n_guns_involved"][i] / df3["entries"][i]
    
    df3["noHarmProp"][i] = df3["noHarm"][i]/df3["entries"][i]
    df3["incidentsWithDeathProp"][i] = df3["incidentsWithDeath"][i]/df3["entries"][i]
    df3["incidentsWithInjuryProp"][i] = df3["incidentsWithInjury"][i]/df3["entries"][i]
    df3["multiGunCountProp"][i] = df3["multiGunCount"][i]/df3["entries"][i]
    
df3[:10]

Number of matching rows for ( date , 2013-01 ): 1
Number of matching rows for ( date , 2013-01 ): 6
Number of matching rows for ( date , 2013-02 ): 1
Number of matching rows for ( date , 2013-02 ): 5


/home/olive/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Number of matching rows for ( date , 2013-03 ): 1
Number of matching rows for ( date , 2013-03 ): 6
Number of matching rows for ( date , 2013-04 ): 1
Number of matching rows for ( date , 2013-04 ): 6
Number of matching rows for ( date , 2013-05 ): 1
Number of matching rows for ( date , 2013-05 ): 12
Number of matching rows for ( date , 2013-06 ): 1
Number of matching rows for ( date , 2013-06 ): 8
Number of matching rows for ( date , 2013-07 ): 1
Number of matching rows for ( date , 2013-07 ): 10
Number of matching rows for ( date , 2013-08 ): 1
Number of matching rows for ( date , 2013-08 ): 9
Number of matching rows for ( date , 2013-09 ): 1
Number of matching rows for ( date , 2013-09 ): 8
Number of matching rows for ( date , 2013-10 ): 1
Number of matching rows for ( date , 2013-10 ): 6
Number of matching rows for ( date , 2013-11 ): 1
Number of matching rows for ( date , 2013-11 ): 5
Number of matching rows for ( date , 2013-12 ): 1
Number of matching rows for ( date , 2013-12 ): 

,date,entries,noHarm,incidentsWithDeath,incidentsWithInjury,multiGunCount,noHarmProp,incidentsWithDeathProp,incidentsWithInjuryProp,multiGunCountProp,killAvg,injAvg,nogunAvg,n_killed,n_injured,n_guns_involved,incident_characteristics:Shot - Wounded/Injured,"incident_characteristics:Shot - Dead (murder, accidental, suicide)",incident_characteristics:Shots Fired - No Injuries,incident_characteristics:Bar/club incident - in or around establishment,incident_characteristics:Suicide^,incident_characteristics:Murder/Suicide,incident_characteristics:Attempted Murder/Suicide (one variable unsuccessful),incident_characteristics:Domestic Violence,"incident_characteristics:Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)","incident_characteristics:Drive-by (car to street, car to car)",incident_characteristics:Gang involvement,incident_characteristics:Home Invasion,incident_characteristics:Home Invasion - Resident killed,incident_characteristics:Officer Involved Incident,incident_characteristics:Officer Involved Shooting - Officer shot,incident_characteristics:Officer Involved Shooting - Officer killed,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator shot,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator suicide at standoff,"incident_characteristics:Spree Shooting (multiple victims, multiple locations)",incident_characteristics:Possession of gun by felon or prohibited person,incident_characteristics:Institution/Group/Business,incident_characteristics:Officer Involved Shooting - subject/suspect/perpetrator killed,incident_characteristics:Animal shot/killed,incident_characteristics:Home Invasion - Resident injured,incident_characteristics:Drug involvement,incident_characteristics:Armed robbery with injury/death and/or evidence of DGU found,"incident_characteristics:Mass Murder (4+ deceased victims excluding the subject/suspect/perpetrator , one location)",incident_characteristics:Concealed Carry License - Perpetrator,incident_characteristics:Possession (gun(s) found during commission of other crimes),incident_characteristics:Stolen/Illegally owned gun{s} recovered during arrest/warrant,incident_characteristics:School Incident,incident_characteristics:School Shooting - university/college,incident_characteristics:Kidnapping/abductions/hostage,incident_characteristics:Car-jacking,incident_characteristics:Hate crime,incident_characteristics:House party,incident_characteristics:Defensive Use,"incident_characteristics:Defensive Use - Crime occurs, victim shoots subject/suspect/perpetrator",incident_characteristics:Defensive Use - Victim stops crime,incident_characteristics:Workplace shooting (disgruntled employee),"incident_characteristics:Assault weapon (AR-15, AK-47, and ALL variants defined by law enforcement)",incident_characteristics:Accidental Shooting,incident_characteristics:Accidental Shooting - Injury,incident_characteristics:Accidental/Negligent Discharge,incident_characteristics:Non-Shooting Incident,incident_characteristics:Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator ),incident_characteristics:ATF/LE Confiscation/Raid/Arrest,incident_characteristics:Unlawful purchase/sale,incident_characteristics:Accidental Shooting - Death,incident_characteristics:BB/Pellet/Replica gun,incident_characteristics:Brandishing/flourishing/open carry/lost/found,incident_characteristics:Child Involved Incident,incident_characteristics:Child injured self,incident_characteristics:Gun range/gun shop/gun show shooting,incident_characteristics:Road rage,incident_characteristics:Gun(s) stolen from owner,incident_characteristics:School Shooting - elementary/secondary school,incident_characteristics:Home Invasion - No death or injury,incident_characteristics:Implied Weapon,incident_characteristics:Accidental Shooting at a Business,incident_characteristics:Self-Inflicted (not suicide or suicide attempt - NO P

In [34]:
df.to_csv('out.csv', index=False)
df1.to_csv('out1.csv', index=False)
#df2.to_csv('out2.csv', index=False)
df3.to_csv('out3.csv', index=False)